In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mutual_info_score,  roc_curve, auc, confusion_matrix, classification_report, roc_auc_score, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV, cross_val_predict, cross_val_score
import matplotlib.pyplot as plt
from mrmr import mrmr_classif
from sklearn.feature_selection import mutual_info_classif
from scipy import stats
from scipy.stats import norm

# Read data
#file path: CT --- RFs_CT_bin50\6-4ADASYN-1; PET (original) --- RFs_PET1_bin0.25\6-4ADASYN-1\original; 
#           PET (pre-Combat or Limma) --- RFs_PET1_bin0.25\6-4ADASYN-1\pre-Combat or Limma; PET (standardized) --- RFs_PET2_bin0.05\6-4ADASYN-1; 
train_data = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_PET1_bin0.25\6-4ADASYN-1\pre-Combat\train.csv')
test_data = pd.read_csv(r'C:\Users\37427\Desktop\github\FS-ML\RFs_PET1_bin0.25\6-4ADASYN-1\pre-Combat\test.csv')

X_train = train_data.iloc[:, 1:] 
y_train = train_data.iloc[:, 0]  
X_test = test_data.iloc[:, 1:]   
y_test = test_data.iloc[:, 0]

# Feature Selector ---- JMIM
def entropy(hist):
    pk = hist / hist.sum()
    return -np.sum(pk[pk > 0] * np.log2(pk[pk > 0]))

def JMIM_single_feature(X, Y):
    X = np.asarray(X).ravel()
    Y = np.asarray(Y).ravel()
    combined = np.concatenate((X, Y))
    thresh = np.percentile(combined, 50)  
    max_JMIM = 0
    
    for threshold in np.linspace(thresh, np.max(X), num=100):
        X_b = (X < threshold).astype(int)
        Y_b = (Y < threshold).astype(int)
        JMIM = mutual_info_score(X_b, Y_b)
        if JMIM > max_JMIM:
            max_JMIM = JMIM
    return max_JMIM

def jmim_feature_selection(X, y, k):
    selected_features = []
    remaining_features = list(range(X.shape[1]))

    while len(selected_features) < k:
        max_jmim = 0
        best_feature_idx = None

        for i in remaining_features:
            jmim = JMIM_single_feature(X.iloc[:, i], y)
            if jmim > max_jmim:
                max_jmim = jmim
                best_feature_idx = i

        if best_feature_idx is not None:
            selected_features.append(best_feature_idx)
            remaining_features.remove(best_feature_idx)

    return selected_features

selected_feature_indices = jmim_feature_selection(X_train, y_train, 10)
selected_features = [X_train.columns[i] for i in selected_feature_indices]
print("Selected features names:", selected_features)
X_train_selected_top_ten = X_train[selected_features]
X_test_selected_top_ten = X_test[selected_features]

Selected features names: ['wavelet.LLH_glszm_GrayLevelVariance', 'log.sigma.1.mm.3D_glcm_Correlation', 'log.sigma.2.mm.3D_glrlm_LowGrayLevelRunEmphasis', 'log.sigma.1.mm.3D_glcm_Imc1', 'wavelet.HLL_glszm_ZonePercentage', 'log.sigma.3.mm.3D_firstorder_Skewness', 'wavelet.LHL_glcm_Correlation', 'wavelet.LLH_glcm_Correlation', 'wavelet.HLH_glcm_InverseVariance', 'log.sigma.3.mm.3D_glszm_SmallAreaEmphasis']


In [2]:
#Classifier --- K-NearestNeighbor
np.random.seed(0)
param_grid = {
    'n_neighbors': list(range(1, 31)),
    'weights': ['uniform', 'distance'],  
    'metric': ['euclidean', 'manhattan', 'minkowski'],  
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  
}
knn = KNeighborsClassifier()
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_selected_top_ten, y_train)
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

y_train_pred_cv = cross_val_predict(best_model, X_train_selected_top_ten, y_train, cv=5)
y_train_pred_proba_cv = cross_val_predict(best_model, X_train_selected_top_ten, y_train, cv=5, method='predict_proba')[:, 1]
train_precision_cv, train_recall_cv, train_f1_cv, _ = precision_recall_fscore_support(y_train, y_train_pred_cv, average='binary')
y_test_pred = best_model.predict(X_test_selected_top_ten)
y_test_pred_proba = best_model.predict_proba(X_test_selected_top_ten)[:, 1]
test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(y_test, y_test_pred, average='binary')

train_accuracy_cv = cross_val_score(best_model, X_train_selected_top_ten, y_train, cv=5, scoring='accuracy').mean()
test_accuracy = accuracy_score(y_test, y_test_pred)
conf_mat_train_cv = confusion_matrix(y_train, y_train_pred_cv)
conf_mat_test = confusion_matrix(y_test, y_test_pred)
sensitivity_train = conf_mat_train_cv[1, 1] / (conf_mat_train_cv[1, 1] + conf_mat_train_cv[1, 0])  # TP / (TP + FN)
sensitivity_test = conf_mat_test[1, 1] / (conf_mat_test[1, 1] + conf_mat_test[1, 0])  # TP / (TP + FN)
specificity_train = conf_mat_train_cv[0, 0] / (conf_mat_train_cv[0, 0] + conf_mat_train_cv[1, 0])
specificity_test = conf_mat_test[0, 0] / (conf_mat_test[0, 0] + conf_mat_test[1, 0])
ppv_train = conf_mat_train_cv[1, 1] / (conf_mat_train_cv[1, 1] + conf_mat_train_cv[0, 1])
npv_train = conf_mat_train_cv[0, 0] / (conf_mat_train_cv[0, 0] + conf_mat_train_cv[1, 0])
ppv_test = conf_mat_test[1, 1] / (conf_mat_test[1, 1] + conf_mat_test[0, 1])
npv_test = conf_mat_test[0, 0] / (conf_mat_test[0, 0] + conf_mat_test[1, 0])
train_precision = precision_score(y_train, y_train_pred_cv)
test_precision = precision_score(y_test, y_test_pred)
train_recall = recall_score(y_train, y_train_pred_cv)
test_recall = recall_score(y_test, y_test_pred)
train_f1 = f1_score(y_train, y_train_pred_cv)
test_f1 = f1_score(y_test, y_test_pred)
print(f"Sensitivity (Cross-Validation): {sensitivity_train:.4f}")
print(f"Specificity (Cross-Validation): {specificity_train:.4f}")
print(f"PPV (Cross-Validation): {ppv_train:.4f}")
print(f"NPV (Cross-Validation): {npv_train:.4f}")
print(f"Precision (Cross-Validation): {train_precision:.4f}")
print(f"Recall (Cross-Validation): {train_recall:.4f}")
print(f"F1 Score (Cross-Validation): {train_f1:.4f}")
print(f"Train Accuracy (Cross-Validation): {train_accuracy_cv:.4f}")
print(f"Sensitivity (Test): {sensitivity_test:.4f}")
print(f"Specificity (Test): {specificity_test:.4f}")
print(f"PPV (Test): {ppv_test:.4f}")
print(f"NPV (Test): {npv_test:.4f}")
print(f"Precision (Test): {test_precision:.4f}")
print(f"Recall (Test): {test_recall:.4f}")
print(f"F1 Score (Test): {test_f1:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

fpr_train, tpr_train, _ = roc_curve(y_train, y_train_pred_proba_cv)
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_pred_proba)
np.random.seed(0)
def bootstrap_roc_auc(y_true, y_scores, n_bootstraps=1000):
    auc_scores = np.zeros(n_bootstraps)
    for i in range(n_bootstraps):
        indices = np.random.choice(len(y_true), size=len(y_true))
        auc_scores[i] = roc_auc_score(y_true[indices], y_scores[indices])
    return auc_scores

auc_scores_train = bootstrap_roc_auc(y_train, y_train_pred_proba_cv)
auc_train_mean = np.mean(auc_scores_train)
auc_train_std = np.std(auc_scores_train)
auc_train_lower = norm.ppf(0.025, loc=auc_train_mean, scale=auc_train_std)
auc_train_upper = norm.ppf(0.975, loc=auc_train_mean, scale=auc_train_std)
print(f"Cross-Validation AUC: {auc_train_mean:.4f} (95% CI: {auc_train_lower:.4f}, {auc_train_upper:.4f})")

auc_scores_test = bootstrap_roc_auc(y_test, y_test_pred_proba)
auc_test_mean = np.mean(auc_scores_test)
auc_test_std = np.std(auc_scores_test)
auc_test_lower = norm.ppf(0.025, loc=auc_test_mean, scale=auc_test_std)
auc_test_upper = norm.ppf(0.975, loc=auc_test_mean, scale=auc_test_std)
print(f"Test AUC: {auc_test_mean:.4f} (95% CI: {auc_test_lower:.4f}, {auc_test_upper:.4f})")

Best parameters: {'algorithm': 'auto', 'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
Sensitivity (Cross-Validation): 0.9072
Specificity (Cross-Validation): 0.8953
PPV (Cross-Validation): 0.7273
NPV (Cross-Validation): 0.8953
Precision (Cross-Validation): 0.7273
Recall (Cross-Validation): 0.9072
F1 Score (Cross-Validation): 0.8073
Train Accuracy (Cross-Validation): 0.7969
Sensitivity (Test): 0.6053
Specificity (Test): 0.7692
PPV (Test): 0.5111
NPV (Test): 0.7692
Precision (Test): 0.5111
Recall (Test): 0.6053
F1 Score (Test): 0.5542
Test Accuracy: 0.6636
Cross-Validation AUC: 0.8040 (95% CI: 0.7526, 0.8554)
Test AUC: 0.6478 (95% CI: 0.5564, 0.7391)
